Tidy Tuesday 12 July 2022 - European Flights

Tidy Tuesday repo for this week: https://github.com/rfordatascience/tidytuesday/tree/master/data/2022/2022-07-12

Airport locations data source: https://github.com/mborsetti/airportsdata

alt text: geographic scatter plot of European airport flights before and during COVID

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
import imageio
import os
import shutil

In [ ]:
flights = pd.read_csv("https://raw.githubusercontent.com/rfordatascience"+\
                          "/tidytuesday/master/data/2022/2022-07-12/flights.csv")
airports = pd.read_csv("https://raw.githubusercontent.com/mborsetti"+\
                           "/airportsdata/main/airportsdata/airports.csv")
airports["APT_ICAO"] = airports["icao"].copy()
df = flights.merge(airports, on=["APT_ICAO"])
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]))
gdf.head()

In [ ]:
if os.path.exists("frames"):
    shutil.rmtree("frames") # delete old frames dir
os.mkdir("frames") # make new frames dir
# [os.remove(f.path) for f in os.scandir(".") if f.name.endswith(".png")]

world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
europe = world[world.continent == "Europe"]


dates = np.unique(gdf["FLT_DATE"])[::7]
filenames = list()
for date in tqdm(dates):

    temp = gdf[gdf["FLT_DATE"]==date]

    fig, ax = plt.subplots(figsize=[20,15])
    plt.title(date.split("T")[0], fontsize=30)
    europe.plot(ax=ax,
                color="lightgreen", 
                edgecolor="black",)
    
    temp.plot(ax=ax, 
#               column="FLT_TOT_1",
              color="indigo",
              markersize=temp["FLT_TOT_1"],)

    plt.xlim([-40,55])
    plt.ylim([25,70])
    plt.axis("off")
    
#     cax = fig.add_axes([0.93, 0.25, 0.03, 0.5])
#     sm = plt.cm.ScalarMappable(cmap="magma", 
#                                norm=plt.Normalize(vmin=gdf["FLT_TOT_1"].min(), 
#                                                   vmax=gdf["FLT_TOT_1"].max()))
#     sm._A = []
#     plt.colorbar(sm, cax)
    
    filename = f"{date.split('T')[0]}.png"
    plt.savefig(os.path.join("frames",filename), 
                bbox_inches="tight",
                dpi=200)
    filenames.append(filename)

    fig.clf()
    plt.close()
    
assert len(filenames) == len(dates) # validation

In [ ]:
frames = np.array([imageio.imread(os.path.join("frames",filename)) for filename in tqdm(filenames)])
imageio.mimsave("european_flights.mp4", frames, fps=10)
imageio.mimsave("european_flights.gif", frames, fps=10)